<div style='width=100%; display:flex;flex-direction:row'><img  src=https://universidadeuropea.com/resources/media/images/universidad-europea-logo_poc9mEM.original.png width=100  style='  margin-left: auto;margin-right: auto; width: 25%; height:25%;'><img  src=https://i.ibb.co/1068C7j/EATEASER.jpg width=100 style='  margin-left: auto;margin-right: auto; width: 10%;height:25%;'></div>

<div style='margin:auto;text-align: center;font-family: "Times New Roman", Times, serif; font-weight: bold;'>PROYECTO COMUTACIONAL<br><br>EATEASER - VOZ A TEXTO</div>

<div style='width:100%; display:flex;flex-direction:row'>
    <div style='width:50%;margin-right:5cm;'>
        <p style='font-family: "Times New Roman", Times, serif; font-weight: bold;'>ESTUDIANTES</p>
<ul style='font-family: "Times New Roman", Times, serif;'>
    <li>Adilem Dobras 21911633</li><li>Roberto Echevarria 21823680</li><li>Carlos Gonzales 22067726</li><li>Juan Carlos Rondeau 21816176</li></ul> </div>

</div>

<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">1. Importamos las librerias</h2>

In [1]:
"""
!pip install pytube
!pip install SpeechRecognition
!pip install pydub
!pip install moviepy
!pip install beautifulsoup4
!pip install nltk
"""

'\n!pip install pytube\n!pip install SpeechRecognition\n!pip install pydub\n!pip install moviepy\n!pip install beautifulsoup4\n!pip install nltk\n'

In [2]:
from pytube import YouTube
import speech_recognition as sr
from pydub import AudioSegment
import moviepy.editor as mp
from bs4 import BeautifulSoup
import glob
import os
import re
from os import listdir
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import requests
from pydub.silence import split_on_silence
import pathlib

C:\Users\thena\.conda\envs\EatEaser\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">2. Inicio del programa</h2>

<h3  style='font-family: "Times New Roman", Times, serif; font-weight: bold;text-align:center;font-size:14px'>CLASE CONTROLADORVIDEO</h3><p style='font-family: "Times New Roman", Times, serif; font-size:14px'>En esta clase se realizará los ajustes para manejar el video recibido y manipularlo.</p>


In [3]:
class ControladorVideo:
    def __init__(self,enlace): 
        self._idvideo = self.reenumerar()
        self.enlacevideo=enlace
        self.yt=YouTube(self.enlacevideo)
        self.nombrevideo=''
        self.titulovideo=self.yt.title
        self.autorvideo=self.yt.author
        self.fechavideo=self.yt.publish_date
        self.duracionvideo=self.yt.length
    """|DESCARGAR VIDEO URL: descarga el video de youtube
       |return: devuelve una ruta absoluta"""
    def descargarVideoURL(self):
        recetasVideos = 'recetasvideos/'
        #aqui creo un nuevo id para el nuevo video
        self._idvideo= self._idvideo+1
        #esta sera el archivo del video y su nuevo nombre
        nombre='receta'+str(self._idvideo)
        #le pedimos al pytube que solo nos descargue el audio y lo descargamos
        t=self.yt.streams.filter(file_extension='mp4').first().download(output_path=recetasVideos,filename=nombre+'.mp4')
        #devolvemos el nombre
        return nombre
    """|PARSEO VIDEO: pasa el video de .mp4 a .wav
       |nombre: es un string que se colocara el nombre del video
       |return: devuelve el nuevo nombre del audio en .wav"""
    def parseoVideo(self,nombre):
        recetasVideos = 'recetasvideos/'
        #tomamos el video en mp4 
        track = mp.VideoFileClip(recetasVideos+nombre+'.mp4')
        #cambiamos el video a .wav
        nombre_wav="{}.wav".format(nombre)
        track.audio.write_audiofile(recetasVideos+nombre_wav)
        track.close()
        return nombre+'.wav'
    def reenumerar(self):
        contador = 0
        for item in pathlib.Path("./recetastextos/").iterdir():
            if item.is_file():
                contador += 1
            elif item.is_dir():
                for subitem in item.iterdir():
                    if subitem.is_file():                     
                        contador += 1
        return contador
    """|SPEECH TEXT:Transforma el audio a texto
       |nombre: es un string que se colocara el nombre del video
       |return: devuelve un string con el texto devuelto"""
    def speech_text(self,nombre):
        recetasVideos = 'recetasvideos/'
        #instanciamos el recognizer
        r = sr.Recognizer()
        audio = sr.AudioFile(recetasVideos+nombre)
        with audio as source:
            audio_file = r.record(source)
        #transcribimos el audio a texto
        result = r.recognize_google(audio_file, language = 'es-ES')
        return result
        

<h3  style='font-family: "Times New Roman", Times, serif; font-weight: bold;text-align:center;font-size:14px'>CLASE DEPURADOR</h3><p style='font-family: "Times New Roman", Times, serif; font-size:14px'>En esta clase se realizará el proceso de extraccion, transformacion y carga de nuestro programa EATEASER.</p>


In [4]:
#si el video es mayor de 3 minutos no funciona
#si el video es en ingles no funciona
class Depurador:
    def __init__(self): 
        self.rec=RecursosAdicionales()
    """|EXTRACCION: proceso etl donde extraemos al informacion del video 
       |enlace: es un string que se colocara el enlace del video"""
    def extraccion(self,enlace):
        try:
            #instanciamos el controlador de videos
            cv=ControladorVideo(enlace)
            #paso 1: descargamos el video
            cv.nombrevideo=cv.descargarVideoURL()
            #paso 2: pasamos el video a .wav
            nombre=cv.parseoVideo(cv.nombrevideo)
            #paso 3: pasamos el .wav a texto
            try:
                result=cv.speech_text(nombre)
                 #paso 4: escribimos el texto recibido en un txt
                resultado=self.rec.escritura(cv.nombrevideo,"Titulo:"+cv.titulovideo+"\n"+"Autor:"+cv.autorvideo+"\n"+"Fecha Publicacion:"+str(cv.fechavideo)+"\n"+"Entradilla:"+result)
                #paso 5: eliminamos los mp4
                self.rec.eliminacionMp4()
                return resultado   
            except:
                print("No se ha podido transcribir el video: "+ cv.nombrevideo + " - "+ cv.titulovideo)
                self.rec.eliminacionMp4()
                return None
        except:
            print("No se ha podido descargar el video: "+ cv.nombrevideo + " - "+ cv.titulovideo)
            return None
          
    def transformacion(self):
        print()
    def carga(self):
        print()

In [5]:
#aqui iran lecturas-escrituras-guardar-eliminar cosas en bases de datos
class RecursosAdicionales:
    """|ESCRITURA: escribe textos txt
       |nombre: nombre del 
       |return: devuelve el audio en texto"""
    #yo siento que en vez de cambiar de carpeta los textos...deberiamos eliminar los mp4
    #y si lo mas factible aqui es que guardemos el video por su nombre?
    #creo que no debemos contar los files porque cansan el programa
    
    def escritura(self,nombre,texto):
        recetasTextos = './recetastextos/'
        if not(os.path.exists(recetasTextos)):
            os.mkdir(recetasTextos)
        f = open(recetasTextos+nombre+'.txt', 'w')
        f.write(texto)
        f = open(recetasTextos+nombre+'.txt', "r")
        print(f.read())
        f.close()
    def eliminacionMp4(self):
        recetasVideos = './recetasvideos/'
        py_files = glob.glob(recetasVideos+'*.mp4')
        for py_file in py_files:
            try:
                os.remove(py_file)
            except OSError as e:
                print(f"Error:{ e.strerror}")      
        

In [6]:
class WebScrap:
    def __init__(self): 
        self.headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    def request(self, url):
        request1 = requests.get(url, headers=self.headers)
        html = request1.content
        soup = BeautifulSoup(html, 'html.parser')
        return soup
    def verificar_alimento(self,alimento):
        soup = self.request( 'https://www.themealdb.com/api/json/v1/1/search.php?s='+alimento)
        print(soup)
        

<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">3. Main</h2>

In [8]:
#y si tambien vemos si le permitimos al usuario que meta videos?
dep=Depurador()
if __name__ == '__main__':
    #dep.extraccion('https://www.youtube.com/watch?v=6PzQY1E2s2g&list=PLxHmjpcgU5ArC2rY5cpoIcZoVKB_0UHfR&ab_channel=PlatosF%C3%A1cilesconTamara')
    #dep.extraccion('https://www.youtube.com/watch?v=PsqR5M8rdjA&list=LL&index=9&t=4s')
    #dep.extraccion('https://www.youtube.com/watch?v=xfYcM_jHgPY')
    #dep.extraccion('https://www.youtube.com/watch?v=wiCfqc5W-yo')
    #error#dep.extraccion('https://www.youtube.com/watch?v=3DnPkf9rP_0')
    #error#dep.extraccion('https://www.youtube.com/watch?v=xVsgKMZFCZY')
    #dep.extraccion('https://www.youtube.com/watch?v=rpCe0RPMY94')
    #dep.extraccion('https://www.youtube.com/watch?v=rv4gLMa-FYk')
    #dep.extraccion('https://www.youtube.com/watch?v=VS8zYxBj4r8')
    #dep.extraccion('https://www.youtube.com/watch?v=o99JXrEkZoo')
    #dep.extraccion('https://www.youtube.com/watch?v=lKkg5L23b3M')
    #dep.extraccion('https://www.youtube.com/watch?v=PsqR5M8rdjA&t=14s')
    #dep.extraccion('https://www.youtube.com/watch?v=IvZaAL6qYe0&t=29s')
    #dep.extraccion('https://www.youtube.com/watch?v=SIMQBuuyE9M')
    #dep.extraccion('https://www.youtube.com/watch?v=_YoZfg7R8Hk')
    #dep.extraccion('https://www.youtube.com/watch?v=Zv7KdlOBk7Y')
    #dep.extraccion('https://www.youtube.com/watch?v=mFcN4btaZyI&t=2s')
    #dep.extraccion('https://www.youtube.com/watch?v=sRmmQBBln9Q')
    #dep.extraccion('https://www.youtube.com/watch?v=-QoTJJJfeEE')
    #dep.extraccion('https://www.youtube.com/watch?v=JRY5obPKPzo&list=PLxHmjpcgU5ArC2rY5cpoIcZoVKB_0UHfR&index=5&ab_channel=PlatosF%C3%A1cilesconTamara')
    #dep.extraccion('https://www.youtube.com/watch?v=stFmx7OCy1k&ab_channel=RecetasdeEsbieta')
    dep.extraccion("https://www.youtube.com/watch?v=qqTqePGIjhc")
    
    
    
    

MoviePy - Writing audio in recetasvideos/receta19.wav


MoviePy - Done.
No se ha podido transcribir el video: receta19 - No comas pan, Desayuno saludable y súper rico - Listo en 10 Minutos
